<a href="https://colab.research.google.com/github/mehedihasanbijoy/BanglaLLMs/blob/main/Text%20Classification/SentNoB_BERT_torch_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Required Libraries

In [54]:
%%capture

!pip install -q gdown
!pip install --upgrade transformers[torch]
!pip install datasets

## Download Necessary Corpora

In [55]:
%%capture

# Download the folder named SentNoB (for fine-tune the LLM)
!gdown "https://drive.google.com/drive/folders/1EjBD0TumnpbFui4EJF7msT7O9Md6qfQw?usp=sharing" --folder

# Fetch the corpus (for fine-tune the tokenizer)
!gdown "https://drive.google.com/drive/folders/1oIT7DZhd4uXTpjgBeRGSP-Fs-1Ux3m-b?usp=sharing" --folder

## Import Libraries

In [56]:
import torch
from datasets import Dataset
from transformers import Trainer, TrainingArguments, AutoTokenizer, BertForSequenceClassification, AdamW
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

## Fine-tune the Tokenizer

In [57]:
test = pd.read_csv("/content/BanglaParaphraseBUETNLP/test.csv")
train = pd.read_csv("/content/BanglaParaphraseBUETNLP/train.csv")
valid = pd.read_csv("/content/BanglaParaphraseBUETNLP/valid.csv")

df = pd.concat([test, train, valid], ignore_index=True)
df.reset_index(drop=True, inplace=True)

source_texts = df['source'].tolist()
target_texts = df['target'].tolist()
all_texts = source_texts + target_texts
# all_texts = all_texts[:500000]
all_texts[:3]

['কিছুদিন আগে প্যারিস থেকে ঘুরে এসেছি।',
 'ভাড়া করে ফেললেন কার্নেগি হলের মতো অত্যন্ত অভিজাত অডিটোরিয়াম, যেখানে হাজার হাজার মানুষ একসাথে বসে পারফর্মেন্স দেখতে পারে।',
 'সম্পূর্ণ নিয়ন্ত্রণ হারিয়ে জাহাজ পড়লো ঘোর সমুদ্রে।']

In [58]:
all_considered_characters = [
    ' ',  'ঁ',  'ং',  'ঃ',  'অ',  'আ',  'ই',  'ঈ',  'উ',  'ঊ',  'ঋ',  'এ',  'ঐ',  'ও',  'ঔ',
    'ক',  'খ',  'গ',  'ঘ',  'ঙ',  'চ',  'ছ',  'জ',  'ঝ',  'ঞ',  'ট',  'ঠ',  'ড',  'ঢ',  'ণ',  'ত',
    'থ',  'দ',  'ধ',  'ন',  'প',  'ফ',  'ব',  'ভ',  'ম',  'য',  'র',  'ল',  'শ',  'ষ',  'স',  'হ',
    'ড়',   'ঢ়',   'য়',  '়',  'া',  'ি',  'ী',  'ু',  'ূ',  'ৃ',  'ে',  'ৈ',  'ো',  'ৌ',  '্',  'ৎ',
    '০',  '১',  '২',  '৩',  '৪',  '৫',  '৬',  '৭',  '৮',  '৯']

In [59]:
all_cleaned_sentences = []

for sent in tqdm(all_texts):
    cleaned_sent = ""
    for char in sent:
        if char in all_considered_characters:
            cleaned_sent += str(char)
    all_cleaned_sentences.append(cleaned_sent)

100%|██████████| 933260/933260 [01:48<00:00, 8610.69it/s] 


In [60]:
pretrained_tokenizer_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_tokenizer_name)

example_sent = all_cleaned_sentences[random.randint(0, len(all_cleaned_sentences)-1)]

print(example_sent)
print(tokenizer.tokenize(example_sent))
print(tokenizer.encode(example_sent))
print(tokenizer.decode(tokenizer.encode(example_sent)))

ফেব্রিক কাটিং এর ক্ষেত্রে লেজার কাটিং সিস্টেম অন্যতম উন্নত প্রযুক্তি
['[UNK]', 'ক', '##া', '##ট', '##ি', '##ং', 'এ', '##র', 'ক', '##ষ', '##ে', '##ত', '##র', '##ে', 'ল', '##ে', '##জ', '##া', '##র', 'ক', '##া', '##ট', '##ি', '##ং', 'স', '##ি', '##স', '##ট', '##ে', '##ম', 'অ', '##ন', '##য', '##ত', '##ম', 'উ', '##ন', '##ন', '##ত', 'প', '##র', '##য', '##ক', '##ত', '##ি']
[101, 100, 1353, 29914, 29895, 29915, 29882, 1351, 29908, 1353, 29911, 29917, 29898, 29908, 29917, 1373, 29917, 29894, 29914, 29908, 1353, 29914, 29895, 29915, 29882, 1376, 29915, 29912, 29895, 29917, 29906, 1347, 29902, 29907, 29898, 29906, 1350, 29902, 29902, 29898, 1367, 29908, 29907, 29889, 29898, 29915, 102]
[CLS] [UNK] কাটিং এর কষেতরে লেজার কাটিং সিসটেম অনযতম উননত পরযকতি [SEP]


In [61]:
# Customize training parameters
vocab_size = 30000
min_frequency = 5

# Fine-tune the tokenizer on your custom dataset
tokenizer_finetuned = tokenizer.train_new_from_iterator(np.array(all_cleaned_sentences).reshape(-1, 1), vocab_size=vocab_size)

print(example_sent)
print(tokenizer_finetuned.tokenize(example_sent))
print(tokenizer_finetuned.encode(example_sent))
print(tokenizer_finetuned.decode(tokenizer_finetuned.encode(example_sent)))

ফেব্রিক কাটিং এর ক্ষেত্রে লেজার কাটিং সিস্টেম অন্যতম উন্নত প্রযুক্তি
['ফেবর', '##িক', 'কাট', '##িং', 'এর', 'কষেতরে', 'লেজার', 'কাট', '##িং', 'সিসটেম', 'অনযতম', 'উননত', 'পরযকতি']
[2, 3381, 149, 3185, 2675, 237, 692, 12107, 3185, 2675, 5141, 1334, 2489, 1929, 3]
[CLS] ফেবরিক কাটিং এর কষেতরে লেজার কাটিং সিসটেম অনযতম উননত পরযকতি [SEP]


## Process the SentNoB Corpus

In [63]:
train_df = pd.read_csv("/content/SentNoB/Train.csv")
val_df = pd.read_csv("/content/SentNoB/Val.csv")
test_df = pd.read_csv("/content/SentNoB/Test.csv")

print(f"Number of instances in training set  : {str(len(train_df)).rjust(4)}")
print(f"Number of instances in validation set: {str(len(val_df)).rjust(4)}")
print(f"Number of instances in Test set      : {str(len(test_df)).rjust(4)}")

Number of instances in training set  : 12575
Number of instances in validation set: 1567
Number of instances in Test set      : 1586


In [65]:
id2label = {0: "Neutral", 1: "Positive", 2: "Negative"}
print(f"id2label: {id2label}")

label2id = {v: k for k, v in id2label.items()}
print(f"label2id: {label2id}")

id2label: {0: 'Neutral', 1: 'Positive', 2: 'Negative'}
label2id: {'Neutral': 0, 'Positive': 1, 'Negative': 2}


In [64]:
def find_len(sent):
    return len(str(sent).split())

train_df['len'] = train_df['Data'].apply(find_len)
test_df['len'] = test_df['Data'].apply(find_len)
val_df['len'] = val_df['Data'].apply(find_len)

train_df = train_df.loc[train_df['len']>5].reset_index(drop=True).iloc[:, :-1]
# test_df = test_df.loc[test_df['len']>5].reset_index(drop=True).iloc[:, :-1]
# val_df = val_df.loc[val_df['len']>5].reset_index(drop=True).iloc[:, :-1]

train_df = train_df.sample(frac=1.)
train_df = train_df.iloc[:int(len(train_df)*0.7), :].reset_index(drop=True)
test_df = train_df.iloc[int(len(train_df)*0.7):int(len(train_df)*0.85), :].reset_index(drop=True)
val_df = train_df.iloc[int(len(train_df)*0.85):int(len(train_df)*1), :].reset_index(drop=True)

# train_df = train_df.rename(columns={'Data': 'text', 'Label': 'label'})
# test_df = test_df.rename(columns={'Data': 'text', 'Label': 'label'})
# val_df = val_df.rename(columns={'Data': 'text', 'Label': 'label'})

print(f"Number of instances in training set  : {str(len(train_df)).rjust(5)}")
print(f"Number of instances in validation set: {str(len(val_df)).rjust(5)}")
print(f"Number of instances in Test set      : {str(len(test_df)).rjust(5)}")

Number of instances in training set  :  8093
Number of instances in validation set:  1214
Number of instances in Test set      :  1214


In [66]:
class CustomCorpus(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df['text'].values[idx]
        ids_n_masks = self.tokenizer(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        return {
            "input_ids": torch.tensor(ids_n_masks["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(ids_n_masks["attention_mask"], dtype=torch.long),
            "targets": torch.tensor(self.df["label"][idx], dtype=torch.long)
        }

In [67]:
max_len = 64
batch_size = 8

In [68]:
train_dataset = CustomCorpus(train_df, tokenizer_finetuned, max_len)
val_dataset = CustomCorpus(val_df, tokenizer_finetuned, max_len)
test_dataset = CustomCorpus(test_df, tokenizer_finetuned, max_len)

In [69]:
train_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 0
                }

In [17]:
training_loader = torch.utils.data.DataLoader(train_dataset, **train_params)
valid_loader = torch.utils.data.DataLoader(val_dataset, **train_params)
testing_loader = torch.utils.data.DataLoader(test_dataset, **test_params)

## Fine-tune BERT

In [70]:
# n_classes = 3
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=n_classes)
# model.to(device)
# print(f"Model is sent to {device}")

In [71]:
from transformers import BertModel

class BERTClass(torch.nn.Module):
    def __init__(self, n_classes):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained("bert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 256)
        self.dropout = torch.nn.Dropout(0.5)
        self.classifier = torch.nn.Linear(256, n_classes)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [72]:
model = BERTClass(n_classes)
model

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [73]:
model.to(device)
print(f"Model is sent to {device}")

Model is sent to cuda


In [74]:
learning_rate = 2e-5

In [75]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=learning_rate)

In [76]:
# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [77]:
def train():
    model.train()
    train_count, correct_preds, train_loss = 0, 0, 0

    for data in tqdm(training_loader):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        # print(outputs.shape, outputs)
        # print(targets.shape, targets)

        optimizer.zero_grad()
        loss = loss_function(outputs, targets)
        # print(loss)
        loss.backward()
        optimizer.step()

        preds_value, preds_idx = torch.max(outputs.data, 1)
        correct_preds += (preds_idx == targets).sum().item()
        train_loss += loss.item() * targets.shape[0]
        train_count += targets.shape[0]

    epoch_acc = (correct_preds / train_count)
    epoch_loss = (train_loss / train_count)

    return epoch_loss, epoch_acc

In [78]:
def valid(data_loader):
    model.eval()
    epoch_loss, correct_preds, total_count = 0, 0, 0
    actual, predictions = [], []

    with torch.no_grad():
        for data in tqdm(data_loader):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)
            preds_value, preds_idx = torch.max(outputs.data, 1)

            correct_preds += (preds_idx == targets).sum().item()
            epoch_loss += loss.item() * targets.shape[0]
            total_count += targets.shape[0]

            # actual.append(preds_idx.detach().cpu().numpy())
            # predictions.append(targets.detach().cpu().numpy())

    epoch_acc = (correct_preds / total_count)
    epoch_loss = (epoch_loss / total_count)

    return epoch_loss, epoch_acc

In [88]:
num_epochs = 100
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1} / {num_epochs}")
    epoch_loss, epoch_acc = train()
    print(f"Training Loss: {epoch_loss}, Training Accuracy: {epoch_acc*100:.2f}%\n")
    epoch_loss, epoch_acc = valid(valid_loader)
    print(f"Validation Loss: {epoch_loss}, Validation Accuracy: {epoch_acc*100:.2f}%\n")

Epoch 1 / 100


100%|██████████| 323/323 [00:37<00:00,  8.55it/s]


Training Loss: 0.9882906758379272, Training Accuracy: 51.78%



100%|██████████| 161/161 [00:05<00:00, 31.90it/s]


Validation Loss: 0.9232338614964763, Validation Accuracy: 61.71%

Epoch 2 / 100


...

Validation Loss: 0.9729529642409388, Validation Accuracy: 54.71%

Epoch 100 / 100


100%|██████████| 323/323 [00:36<00:00,  8.83it/s]


Training Loss: 0.9937844394530305, Training Accuracy: 52.13%



100%|██████████| 161/161 [00:05<00:00, 32.03it/s]

Validation Loss: 0.9763229002748482, Validation Accuracy: 54.71%



In [89]:
def violence_identifier(sentence):
    input_tokens = tokenizer_finetuned(sentence, return_tensors="pt", padding=True, truncation=True)
    input_tokens = {key: value.to(device) for key, value in input_tokens.items()}

    outputs = model(input_tokens['input_ids'], input_tokens['attention_mask'])
    big_val, big_idx = torch.max(outputs.data, dim=1)

    print(f"{'*'*50}\nSentence  : {test_example}\nPrediction: {id2label[big_idx.item()]}\n{'*'*50}")

In [90]:
# Example sentence
test_example = "কুত্তার বাচ্চা দেখতে শুয়োরের বাচ্চার থেকে সুন্দর।"

violence_identifier(test_example)

**************************************************
Sentence  : কুত্তার বাচ্চা দেখতে শুয়োরের বাচ্চার থেকে সুন্দর।
Prediction: Neutral
**************************************************
